In [1]:
from enum import Enum
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
train_inputs = []
train_outputs = []
test_inputs = []
test_outputs = []

#Enum for all the categories and their column number
class Col(Enum):
    Gender = 1
    Menstrual = 2 
    SleepPrior = 3 
    Prescription = 4 
    OralAntibiotics = 5 
    TopAntibiotics = 6 
    Eyes = 7 
    Hands = 8 
    Race = 9 
    Feeling = 10 
    Symptoms = 11 
    Facial = 12 
    LastVomit = 13 
    EnvAllergies = 14 
    Snore = 15 
    GrindTeeth = 16 
    EarInfect = 17 
    AllergyTime = 18 
    Skin = 19 
    OralSurgery = 20 
    HeartDisease = 21 
    BadTeeth = 22 
    EatPrior = 23 
    DrinkPrior = 24 
    GumDay = 25 
    YogurtWeek = 26 
    TeaWeek = 27 
    CoffeeWeek = 28 
    SodaWeek = 29 
    FastFoodWeek = 30 
    EatOutWeek = 31 
    Straw = 32 
    AddedSugar = 33 
    Spicy = 34 
    ShareDrink = 35 
    MeatWeek = 36 
    FreshWeek = 37 
    BrushTongue = 38 
    BrushTeethWeek = 39 
    TobaccoWeek = 40 
    FlossWeek = 41 
    DentistYear = 42 
    Toothbrush = 43 
    Mouthwash = 44 
    Toothpaste = 45 
    WashHandsEat = 46 
    WashHandsRestroom = 47 
    ChangeToothbrush = 48 
    BiteNails = 49 
    OrthoDevice = 50 
    Device = 51 
    Stress = 52 
    HandleStress = 53 
    Organized = 54 
    RateStress = 55 
    AvgSleep = 56 
    AvgWork = 57 
    Setting = 58 
    Environment = 59 
    UpDown = 60 
    Weight = 61 
    ExerciseMonth = 62 
    Transportation = 63 
    Orientation = 64 
    NearDogsWeek = 65 
    NearCatsWeek = 66 
    ShareBathroom = 67 
    BooksYear = 68 
    Alarms = 69 
    Patience = 70 
    Lipstick = 71 
    IntroExtro = 72 
    Relgious = 73 
    OptiPessi = 74 
    SexuallyActive = 75 
    OralSex = 76 
    Music = 77 
    AvgGrade = 78 
    RateHappy = 79 
    Arts = 80 
    Kiss = 81 
    Roommates = 82 

    
#Enum for type of test
class Test(Enum):
    Ttest = 1
    Anova = 2
    

def sleepPrior(category, data):
    if category != Col.SleepPrior:
        return data
    
    #if SleepPrior, manipulate the data so that the subcategories don't go over 7
    #also create new subcategories to make it more general
    data[category.name] = data[category.name].replace(['0', '1', '2', '3'], 'less than 4')
    data[category.name] = data[category.name].replace(['4', '5', '6', '7'], '4 to 7')
    data[category.name] = data[category.name].replace(['8', '9', '10', '11', '12'], 'more than 7')
    return data


def prescription(category, data):
    if category != Col.Prescription:
        return data
    
    #manipulate the data so that if the prescription column has a specific prescription, replace with yes
    data[category.name] = data[category.name].replace(['None', 'none', 'no '], 'no')
    data.loc[(data[category.name] != 'no') & (data[category.name] != 'No') & (data[category.name] != 'nan'), 
             category.name] = 'yes'
    return data


def eyes(category, data):
    if category != Col.Eyes:
        return data
    
    #manipulate the data to make it more generic
    data[category.name] = data[category.name].replace('Green/blue/gold mix', 'hazel')
    return data


def perWeek(category, data):
    if category not in (Col.TeaWeek, Col.CoffeeWeek, Col.SodaWeek, Col.FastFoodWeek, Col.BrushTeethWeek, 
                        Col.FlossWeek, Col.BooksYear, Col.GumDay, Col.EatOutWeek, Col.AvgSleep, Col.AvgWork, 
                        Col.ExerciseMonth) :
        return data
    
    #manipulate the data so results don't show as dates
    data[category.name] = data[category.name].replace('5-Jan', '1-5')
    data[category.name] = data[category.name].replace('10-Jun', '6-10')
    data[category.name] = data[category.name].replace('15-Nov', '11-15')
    
    data[category.name] = data[category.name].replace('2-Jan', '1-2')
    data[category.name] = data[category.name].replace('4-Mar', '3-4')
    data[category.name] = data[category.name].replace('6-May', '5-6')
    data[category.name] = data[category.name].replace('8-Jul', '7-8')
    data[category.name] = data[category.name].replace('10-Sep', '9-10')
    data[category.name] = data[category.name].replace('12-Nov', '11-12')
    
    if category == Col.FlossWeek:
        data[category.name] = data[category.name].replace('5-Nov', '11-15')
    
    return data


def numToWords(category, data):
    if category not in (Col.AddedSugar, Col.Straw, Col.Lipstick):
        return data
    
    #manipulate the data so results don't show as dates
    data[category.name] = data[category.name].replace('1', 'never')
    data[category.name] = data[category.name].replace('2', 'rarely')
    data[category.name] = data[category.name].replace('3', 'sometimes')
    data[category.name] = data[category.name].replace('4', 'often')
    data[category.name] = data[category.name].replace('5', 'always')
    
    return data


def mouthwash(category, data):
    if category != Col.Mouthwash:
        return data
    
    #manipulate the data so that results are in form of yes or no
    data.loc[(data[category.name] != 'never') & (data[category.name] != 'No') & (data[category.name] != 'nan'), 
             category.name] = 'yes'
    data[category.name] = data[category.name].replace('never', 'no')
    return data


def religious(category, data):
    if category != Col.Relgious:
        return data
    
    #manipulate the data so that results are in form of yes or no
    data.loc[(data[category.name] != 'No') & (data[category.name] != 'nan'), category.name] = 'yes'
    data[category.name] = data[category.name].replace('never', 'no')
    return data


def orientation(category, data):
    if category != Col.Orientation:
        return data
    
    #fix spelling mistakes, change bicurious to bisexual to generalize data
    data[category.name] = data[category.name].replace('hetrosexual', 'heterosexual')
    data[category.name] = data[category.name].replace(['bicurious', 'Bicurious'], 'bisexual')
    return data


bactClassDict = {}
total = 0
sumCat = 0
    
def runTests(data, category, outputMap, showAllClasses):
    subcategory = []
    categoryDF = []
    bactClasses = []
    sigClasses = []
    inputs = []
    global total
    global sumCat
    
    #decided not to do these groups
    if category.name in ('Menstrual', 'Race', 'Feeling', 'Symptoms', 'EnvAllergies', 'Skin', 'EatPrior',
                        'DrinkPrior', 'TobaccoWeek', 'Toothpaste', 'BiteNails', 'Transportation', 'Environment', 
                        'NearDogsWeek', 'NearCatsWeek'):
        return False
    
    
    # fix the inconsistant data
    data = sleepPrior(category, data)
    data = prescription(category, data)
    data = eyes(category, data)
    data = perWeek(category, data)
    data = numToWords(category, data)
    data = mouthwash(category, data)
    data = religious(category, data)
    data = orientation(category, data)

    #change the category into its own separate dataframe

    #get list of subcategories
    try:
        data[category.name] = data[category.name].str.lower()
    except:
        data[category.name]
    subcategory = data[category.name].drop_duplicates().values.tolist()
    
    #remove nans (where people didn't put in an answer)
    try:
        subcategory = [subcat.lower().strip() for subcat in subcategory if str(subcat) != 'nan' 
                   and str(subcat) != 'don\'t know' and str(subcat) != 'do not know' and str(subcat) != 'don?t know']
    except:
        subcategory = [subcat for subcat in subcategory if str(subcat) != 'nan' 
                   and str(subcat) != 'don\'t know' and str(subcat) != 'do not know' and str(subcat) != 'don?t know']
        
    #remove any more duplicates
    subcategory = list(set(subcategory))

    #remove outliers/format the data for specific categories
    if category == Col.Spicy:
        subcategory.remove('sometimes')
    if category == Col.Hands:
        subcategory.remove('ambidextrous')
    if category == Col.BrushTeethWeek:
        subcategory.remove('42689');
    

    #decide on which test to use
    if len(subcategory) == 2:
        test = Test.Ttest
    else:
        test = Test.Anova
        
    #load dataframes for each subcategory of all bacterial classes\n",
    for x in range(0, len(subcategory)):
        categoryDF.append(data[data[category.name] == subcategory[x]].iloc[:, 83:125])
        
    print(category.name)
    print(subcategory)
        
    #go through all bacterial classes to perform tests on them
    sigClassCount = 0
    total += 1
    markCat = False
    for x in range(0, 42):
        
        #perform t-test if applicable
        if test == Test.Ttest:
            ttest = ttest_ind(categoryDF[0].iloc[:, x],categoryDF[1].iloc[:, x])
            pvalue = ttest[1]
            
        #perform anova test is applicable
        if test == Test.Anova:
            #keep track of how many subcategories there are to put into anova test
            arg = []
            for y in range(0, len(subcategory)):
                arg.append(categoryDF[y].iloc[:, x])
            anova = stats.f_oneway(*(a for a in arg))
            pvalue = anova[1]

        #keep a list of all bacteria classes and their index
        if len(bactClasses) != 42:
            bactClasses.append(categoryDF[0].iloc[:,x].name)

        #print classes that are significant
        if pvalue <= 0.05 or showAllClasses:
            markCat = True
            #keep track of significant classes
            sigClassCount += 1
            className = categoryDF[0].iloc[:,x].name
            if className not in bactClassDict:
                bactClassDict[className] = 1
            else:
                bactClassDict[className] += 1
            sigClasses.append(className)

            print(categoryDF[0].iloc[:,x].name, "\nP-value: {:4.6f}".format(pvalue))
    
    if markCat:
        sumCat += 1

    # print out info for categories with high numbers of significant classes  
#     if sigClassCount >= 4: 
#         catData = []
#         print(sigClassCount)
#         print(sigClasses)
#         for subcat in data[category.name]:
#             if subcat in subcategory:
#                 catData.append(subcat)
#         print(len(catData))
#         print(catData)
    
    print('\n')
        
    #get and set inputs, key = bactClass, value = list of all values under that class - Design 1
#     for c in sigClasses:
#         classValue = []
#         for index, row in data.iterrows():
#             if row[category.name] in subcategory:
#                 classValue.append(row[c])
#         inputs[c] = classValue
#     print(inputs)


    # UNCOMMENT BELOW HERE FOR NN TESTING

    #get and set inputs Design 2
    for index, row in data.iterrows():
        classValue = []
        for c in sigClasses:
            if row[category.name] in subcategory:
                classValue.append(row[c])
        if row[category.name] in subcategory:
            inputs.append(classValue)
    print(len(inputs), inputs)
    
    #add subcategories to outputMap
    outputMap[category.name] = subcategory
#     print(outputMap)
    
    #get and set outputs so that it looks like [a,b,c]
    outputs = []
    for index, row in data.iterrows():
        if row[category.name] in subcategory:
            outputs.append(row[category.name])
#     print(len(outputs), outputs)
    
    #replace string outputs with numbers
    if outputs and isinstance(outputs[0], str):
        for i in range(0, len(outputs)):
            outputs[i] = subcategory.index(outputs[i])
#     print(len(outputs), outputs)
    
    #separate training and testing data
    for i in range(0, len(inputs)):
        if i % 2 == 1:
            train_inputs.append(inputs[i])
            train_outputs.append(outputs[i])
        else:
            test_inputs.append(inputs[i])
            test_outputs.append(outputs[i])  
    
    

class Main():
    data = pd.read_csv('MasterSheet.csv')
    outputMap = {}
    showAllClasses = False
    
    runTests(data, Col.Gender, outputMap, showAllClasses)
    
#     for x in range(1, 82):
#         runTests(data, Col(x), outputMap, showAllClasses)
        
    #print out the highest appearing sig class - information tim requested
#     sort = sorted(bactClassDict.items(), key=lambda x: x[1], reverse=True)
#     print(sort)
    
    print("sig categories ", repr(sumCat), "/", repr(total))


Gender
['male', 'female']
Bacilli 
P-value: 0.040059
Fusobacteria 
P-value: 0.000003
Epsilonproteobacteria 
P-value: 0.021050
Alphaproteobacteria 
P-value: 0.038678
Thermotogae 
P-value: 0.012158


106 [[3.9442973119999998, 0.076904877, -0.33510018199999997, -0.342277971, -0.353762432], [3.9723758339999997, 0.190906131, -0.362045826, -0.357471775, -0.381358487], [3.760585735, 0.602311415, -0.370152097, -0.361497044, -0.376256186], [3.114230698, -0.22210792399999998, -0.339227998, -0.318056067, -0.331408263], [5.343307545, 0.134594481, -0.308472788, -0.30015785300000003, -0.319163419], [3.503269346, 0.480379028, -0.363317868, -0.366886806, -0.377593619], [3.746173966, -0.021308512, -0.35812048399999996, -0.329250886, -0.343685685], [2.187348146, -0.207054766, -0.33773129700000004, -0.328490128, -0.336431133], [3.77699665, 0.203021108, -0.31253190000000003, -0.35530704, -0.35530704], [4.293834467, -0.064573619, -0.349397739, -0.333604432, -0.351576126], [4.398598967, 0.094596186, -0.3224

In [82]:
# following https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/_index.ipynb
# as a tutorial

import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential()
model.add(layers.Dense(units=4,input_shape=(5,)))
model.add(layers.Dense(64, activation='selu'))
model.add(layers.Dropout (0.3))
model.add(layers.Dense(64, activation='selu'))
model.add(layers.Dense(64, activation='selu'))
model.add(layers.Dropout (0.3))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='Adamax',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(np.array(train_inputs), np.array(train_outputs), epochs=500,
              validation_data=(np.array(test_inputs), np.array(test_outputs)))

model.evaluate(np.array(test_inputs), np.array(test_outputs))

Train on 53 samples, validate on 53 samples
Epoch 1/500
53/53 [==============================] - 5s 92ms/step - loss: 2.6173 - acc: 0.1321 - val_loss: 1.3188 - val_acc: 0.3585
Epoch 2/500
53/53 [==============================] - 0s 320us/step - loss: 1.4865 - acc: 0.4528 - val_loss: 1.0011 - val_acc: 0.5094
Epoch 3/500
53/53 [==============================] - 0s 395us/step - loss: 1.1818 - acc: 0.4340 - val_loss: 0.9077 - val_acc: 0.5472
Epoch 4/500
53/53 [==============================] - 0s 489us/step - loss: 1.1858 - acc: 0.5094 - val_loss: 0.8344 - val_acc: 0.5849
Epoch 5/500
53/53 [==============================] - 0s 433us/step - loss: 1.1181 - acc: 0.4151 - val_loss: 0.7545 - val_acc: 0.5849
Epoch 6/500
53/53 [==============================] - 0s 452us/step - loss: 1.0436 - acc: 0.4906 - val_loss: 0.7101 - val_acc: 0.6226
Epoch 7/500
53/53 [==============================] - 0s 414us/step - loss: 0.7787 - acc: 0.6226 - val_loss: 0.6774 - val_acc: 0.6604
Epoch 8/500
53/53 [=======

Epoch 62/500
53/53 [==============================] - 0s 414us/step - loss: 0.7282 - acc: 0.6226 - val_loss: 0.5993 - val_acc: 0.6981
Epoch 63/500
53/53 [==============================] - 0s 602us/step - loss: 0.7422 - acc: 0.5849 - val_loss: 0.6004 - val_acc: 0.6981
Epoch 64/500
53/53 [==============================] - 0s 376us/step - loss: 0.6659 - acc: 0.6415 - val_loss: 0.6001 - val_acc: 0.6981
Epoch 65/500
53/53 [==============================] - ETA: 0s - loss: 0.5867 - acc: 0.656 - 0s 414us/step - loss: 0.6711 - acc: 0.6038 - val_loss: 0.5995 - val_acc: 0.6981
Epoch 66/500
53/53 [==============================] - 0s 546us/step - loss: 0.5758 - acc: 0.7170 - val_loss: 0.5992 - val_acc: 0.6981
Epoch 67/500
53/53 [==============================] - 0s 546us/step - loss: 0.6370 - acc: 0.6792 - val_loss: 0.6003 - val_acc: 0.6792
Epoch 68/500
53/53 [==============================] - 0s 376us/step - loss: 0.5968 - acc: 0.7170 - val_loss: 0.6023 - val_acc: 0.6604
Epoch 69/500
53/53 [====

Epoch 123/500
53/53 [==============================] - 0s 527us/step - loss: 0.6331 - acc: 0.6415 - val_loss: 0.6014 - val_acc: 0.6604
Epoch 124/500
53/53 [==============================] - 0s 452us/step - loss: 0.6049 - acc: 0.7170 - val_loss: 0.6007 - val_acc: 0.6604
Epoch 125/500
53/53 [==============================] - 0s 452us/step - loss: 0.5756 - acc: 0.6792 - val_loss: 0.5993 - val_acc: 0.6604
Epoch 126/500
53/53 [==============================] - 0s 489us/step - loss: 0.6792 - acc: 0.6226 - val_loss: 0.5987 - val_acc: 0.6604
Epoch 127/500
53/53 [==============================] - 0s 433us/step - loss: 0.5453 - acc: 0.7736 - val_loss: 0.5973 - val_acc: 0.6981
Epoch 128/500
53/53 [==============================] - 0s 470us/step - loss: 0.6361 - acc: 0.7170 - val_loss: 0.5980 - val_acc: 0.6981
Epoch 129/500
53/53 [==============================] - 0s 470us/step - loss: 0.6078 - acc: 0.6981 - val_loss: 0.5994 - val_acc: 0.6604
Epoch 130/500
53/53 [==============================] - 

Epoch 184/500
53/53 [==============================] - 0s 395us/step - loss: 0.5393 - acc: 0.7547 - val_loss: 0.6002 - val_acc: 0.6604
Epoch 185/500
53/53 [==============================] - 0s 470us/step - loss: 0.5822 - acc: 0.6792 - val_loss: 0.5976 - val_acc: 0.6792
Epoch 186/500
53/53 [==============================] - 0s 546us/step - loss: 0.6197 - acc: 0.5849 - val_loss: 0.5974 - val_acc: 0.6981
Epoch 187/500
53/53 [==============================] - 0s 452us/step - loss: 0.5854 - acc: 0.6981 - val_loss: 0.5972 - val_acc: 0.6981
Epoch 188/500
53/53 [==============================] - 0s 452us/step - loss: 0.5372 - acc: 0.7170 - val_loss: 0.5974 - val_acc: 0.6792
Epoch 189/500
53/53 [==============================] - 0s 508us/step - loss: 0.6246 - acc: 0.6415 - val_loss: 0.5976 - val_acc: 0.6792
Epoch 190/500
53/53 [==============================] - 0s 564us/step - loss: 0.5828 - acc: 0.6792 - val_loss: 0.5969 - val_acc: 0.6792
Epoch 191/500
53/53 [==============================] - 

Epoch 245/500
53/53 [==============================] - 0s 470us/step - loss: 0.6049 - acc: 0.6415 - val_loss: 0.5974 - val_acc: 0.6981
Epoch 246/500
53/53 [==============================] - 0s 357us/step - loss: 0.5363 - acc: 0.6981 - val_loss: 0.5996 - val_acc: 0.6792
Epoch 247/500
53/53 [==============================] - 0s 583us/step - loss: 0.6064 - acc: 0.6981 - val_loss: 0.6030 - val_acc: 0.6604
Epoch 248/500
53/53 [==============================] - 0s 564us/step - loss: 0.6062 - acc: 0.6981 - val_loss: 0.6018 - val_acc: 0.6604
Epoch 249/500
53/53 [==============================] - 0s 715us/step - loss: 0.5615 - acc: 0.6792 - val_loss: 0.6016 - val_acc: 0.6792
Epoch 250/500
53/53 [==============================] - 0s 790us/step - loss: 0.5872 - acc: 0.6792 - val_loss: 0.6012 - val_acc: 0.6792
Epoch 251/500
53/53 [==============================] - 0s 565us/step - loss: 0.6054 - acc: 0.6981 - val_loss: 0.6008 - val_acc: 0.6792
Epoch 252/500
53/53 [==============================] - 

53/53 [==============================] - 0s 546us/step - loss: 0.5633 - acc: 0.6981 - val_loss: 0.5970 - val_acc: 0.6792
Epoch 306/500
53/53 [==============================] - 0s 452us/step - loss: 0.5733 - acc: 0.6415 - val_loss: 0.5974 - val_acc: 0.6792
Epoch 307/500
53/53 [==============================] - 0s 414us/step - loss: 0.6085 - acc: 0.5472 - val_loss: 0.5989 - val_acc: 0.6792
Epoch 308/500
53/53 [==============================] - 0s 659us/step - loss: 0.5319 - acc: 0.7170 - val_loss: 0.5998 - val_acc: 0.6792
Epoch 309/500
53/53 [==============================] - 0s 527us/step - loss: 0.5568 - acc: 0.7170 - val_loss: 0.6012 - val_acc: 0.6604
Epoch 310/500
53/53 [==============================] - 0s 527us/step - loss: 0.5489 - acc: 0.6981 - val_loss: 0.6015 - val_acc: 0.6604
Epoch 311/500
53/53 [==============================] - 0s 602us/step - loss: 0.5199 - acc: 0.6604 - val_loss: 0.6013 - val_acc: 0.6604
Epoch 312/500
53/53 [==============================] - 0s 470us/step 

Epoch 366/500
53/53 [==============================] - 0s 489us/step - loss: 0.5699 - acc: 0.6415 - val_loss: 0.6035 - val_acc: 0.6604
Epoch 367/500
53/53 [==============================] - 0s 376us/step - loss: 0.5456 - acc: 0.6792 - val_loss: 0.6049 - val_acc: 0.6604
Epoch 368/500
53/53 [==============================] - 0s 414us/step - loss: 0.5744 - acc: 0.7358 - val_loss: 0.6055 - val_acc: 0.6604
Epoch 369/500
53/53 [==============================] - 0s 489us/step - loss: 0.5855 - acc: 0.6604 - val_loss: 0.6052 - val_acc: 0.6604
Epoch 370/500
53/53 [==============================] - 0s 489us/step - loss: 0.5924 - acc: 0.6415 - val_loss: 0.6011 - val_acc: 0.6792
Epoch 371/500
53/53 [==============================] - 0s 546us/step - loss: 0.5081 - acc: 0.7170 - val_loss: 0.5989 - val_acc: 0.6792
Epoch 372/500
53/53 [==============================] - 0s 659us/step - loss: 0.5484 - acc: 0.7358 - val_loss: 0.5980 - val_acc: 0.6792
Epoch 373/500
53/53 [==============================] - 

53/53 [==============================] - 0s 433us/step - loss: 0.6096 - acc: 0.6415 - val_loss: 0.6009 - val_acc: 0.6981
Epoch 427/500
53/53 [==============================] - 0s 470us/step - loss: 0.5438 - acc: 0.6981 - val_loss: 0.6001 - val_acc: 0.6981
Epoch 428/500
53/53 [==============================] - 0s 489us/step - loss: 0.5265 - acc: 0.6415 - val_loss: 0.6006 - val_acc: 0.6792
Epoch 429/500
53/53 [==============================] - 0s 490us/step - loss: 0.5781 - acc: 0.6604 - val_loss: 0.6038 - val_acc: 0.6792
Epoch 430/500
53/53 [==============================] - 0s 508us/step - loss: 0.5371 - acc: 0.7358 - val_loss: 0.6098 - val_acc: 0.6604
Epoch 431/500
53/53 [==============================] - 0s 489us/step - loss: 0.5770 - acc: 0.6792 - val_loss: 0.6115 - val_acc: 0.6604
Epoch 432/500
53/53 [==============================] - 0s 508us/step - loss: 0.5488 - acc: 0.6981 - val_loss: 0.6114 - val_acc: 0.6604
Epoch 433/500
53/53 [==============================] - 0s 452us/step 

Epoch 487/500
53/53 [==============================] - 0s 489us/step - loss: 0.5041 - acc: 0.7736 - val_loss: 0.6018 - val_acc: 0.6981
Epoch 488/500
53/53 [==============================] - 0s 602us/step - loss: 0.5737 - acc: 0.6226 - val_loss: 0.6020 - val_acc: 0.6981
Epoch 489/500
53/53 [==============================] - 0s 546us/step - loss: 0.5922 - acc: 0.6038 - val_loss: 0.6025 - val_acc: 0.6981
Epoch 490/500
53/53 [==============================] - 0s 546us/step - loss: 0.5734 - acc: 0.6604 - val_loss: 0.6034 - val_acc: 0.6792
Epoch 491/500
53/53 [==============================] - 0s 545us/step - loss: 0.5331 - acc: 0.6415 - val_loss: 0.6033 - val_acc: 0.6792
Epoch 492/500
53/53 [==============================] - 0s 546us/step - loss: 0.6043 - acc: 0.6415 - val_loss: 0.6026 - val_acc: 0.6981
Epoch 493/500
53/53 [==============================] - 0s 546us/step - loss: 0.6027 - acc: 0.6038 - val_loss: 0.6025 - val_acc: 0.6981
Epoch 494/500
53/53 [==============================] - 

[0.6024584432817855, 0.6981132142948654]